In [2]:
library(ArchR)

Loading required package: ggplot2

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins,
    rowOrderStats

In [3]:
set.seed(2022)
addArchRThreads(threads = 1) 
addArchRGenome("hg38")

Setting default number of Parallel threads to 1.

Warning message:
“no function found corresponding to methods exports from ‘BSgenome’ for: ‘releaseName’”
Setting default genome to Hg38.



In [6]:
inputFiles = "/project/scATAC_analysis/scATAcat_review/data/Granja2019/BMMC_D6T1/01_prepare_fragments_file/final_fragment_files/GSM4138889_scATAC_BMMC_D6T1_liftOverhg38_no_chrMt_sorted_fragments.tsv.gz"

In [11]:
ArrowFiles <- createArrowFiles(
  inputFiles = inputFiles,
  sampleNames = "BMMC_D6T1",
  minTSS = 4, #Dont set this too high because you can always increase later
  minFrags = 1000, 
  addTileMat = TRUE,
  addGeneScoreMat = TRUE
)

Using GeneAnnotation set by addArchRGenome(Hg38)!

Using GeneAnnotation set by addArchRGenome(Hg38)!

ArchR logging to : ArchRLogs/ArchR-createArrows-11dc34864ebe7-Date-2024-05-11_Time-23-08-28.log
If there is an issue, please report to github with logFile!

Cleaning Temporary Files

2024-05-11 23:08:33 : Batch Execution w/ safelapply!, 0 mins elapsed.

(BMMC_D6T1 : 1 of 1) Determining Arrow Method to use!

2024-05-11 23:08:33 : (BMMC_D6T1 : 1 of 1) Reading In Fragments from inputFiles (readMethod = tabix), 0.008 mins elapsed.

2024-05-11 23:08:33 : (BMMC_D6T1 : 1 of 1) Tabix Bed To Temporary File, 0.009 mins elapsed.

Warning message in sprintf("%s Reading TabixFile %s Percent", prefix, round(100 * :
“one argument not used by format '%s Reading TabixFile %s Percent'”
2024-05-11 23:10:14 : (BMMC_D6T1 : 1 of 1) Reading TabixFile 8 Percent, 1.693 mins elapsed.

Warning message in sprintf("%s Reading TabixFile %s Percent", prefix, round(100 * :
“one argument not used by format '%s Reading

In [12]:
proj1 <- ArchRProject(
  ArrowFiles = ArrowFiles, 
  outputDirectory = "BMMC_D6T1",
  copyArrows = TRUE #This is recommened so that you maintain an unaltered copy for later usage.
)

Using GeneAnnotation set by addArchRGenome(Hg38)!

Using GeneAnnotation set by addArchRGenome(Hg38)!

Validating Arrows...

Getting SampleNames...

1 


Copying ArrowFiles to Ouptut Directory! If you want to save disk space set copyArrows = FALSE

1 


Getting Cell Metadata...

1 


Merging Cell Metadata...

Initializing ArchRProject...


                                                   / |
                                                 /    \
            .                                  /      |.
            \\\                              /        |.
              \\\                          /           `|.
                \\\                      /              |.
                  \                    /                |\
                  \\#####\           /                  ||
                ==###########>      /                   ||
                 \\##==......\    /                     ||
            ______ =       =|__ /__                     ||      \\\
        ,--'

In [14]:
getAvailableMatrices(proj1)

[1] "GeneScoreMatrix" "TileMatrix"

In [15]:
out_path = "/project/scATAC_analysis/scATAcat_review/data/Granja2019/BMMC_D6T1/05_peaks_and_genesScore_matrix_ArchR/input_for_EpiAnno/" # where to save resulting matrices | with / at the end
threads_=1 # how many threads to use

proj1 <- addIterativeLSI(
    ArchRProj = proj1,
    useMatrix = "TileMatrix", 
    name = "IterativeLSI", 
    iterations = 2, 
    clusterParams = list( #See Seurat::FindClusters
        resolution = c(0.2), 
        sampleCells = 10000, 
        n.start = 10
    ), 
    varFeatures = 25000, 
    dimsToUse = 1:30
)

proj1 <- addClusters(
    input = proj1,
    reducedDims = "IterativeLSI",
    method = "Seurat",
    name = "Clusters",
    resolution = 0.8
)

proj1 <- addUMAP(
    ArchRProj = proj1, 
    reducedDims = "IterativeLSI", 
    name = "UMAP", 
    nNeighbors = 30, 
    minDist = 0.5, 
    metric = "cosine"
)

HDF5_USE_FILE_LOCKING=FALSE
RHDF5_USE_FILE_LOCKING=FALSE

proj2 <- addGroupCoverages(ArchRProj = proj1, groupBy = "Clusters")

pathToMacs2 <- findMacs2()

proj2 <- addReproduciblePeakSet(
    ArchRProj = proj2, 
    groupBy = "Clusters", 
    pathToMacs2 = pathToMacs2
)


#saveArchRProject(ArchRProj = proj2, outputDirectory = "Save-Proj2_afterPeakCalling", load = FALSE)

proj3 <- addPeakMatrix(proj2)

peakMatrix <- getMatrixFromProject(proj3, useMatrix='PeakMatrix')

PeakMatrix_proj3_dense =  as.matrix(peakMatrix@assays@data@listData$PeakMatrix)

write.csv(PeakMatrix_proj3_dense, file = paste(out_path,"PeakMatrix_dense.csv",sep="" )) 

# get the peak regions
proj3_peak_metadata  = (getPeakSet(proj3))
proj3_peak_regions = data.frame(seqnames = seqnames(proj3_peak_metadata),
           start= start(proj3_peak_metadata),
            end = end(proj3_peak_metadata))

write.csv(proj3_peak_metadata, file =paste(out_path,"peak_metadata.csv",sep="" ))
write.csv(proj3_peak_regions, file =paste(out_path,"peak_regions.csv",sep="" ))
write.table(proj3_peak_regions, file =paste(out_path,"peak_regions.tsv",sep="" ),col.names = F,row.names=FALSE,sep="\t", quote = FALSE)

# save the peak matrix as sparse mtx


# save sparse matrix
sparse_matrix <- Matrix(PeakMatrix_proj3_dense , sparse = T )
writeMM(obj = sparse_matrix, file = paste(out_path,"ArchR_peakMatrix.mtx",sep="" ))

# save genes and cells names
combined_peak_features =  paste(proj3_peak_regions$seqnames,"_",proj3_peak_regions$start,"_",proj3_peak_regions$end, sep="")

write.table(as.data.frame(combined_peak_features), file = paste(out_path,"ArchR_peaks.tsv",sep="" ) , row.names = FALSE, quote = FALSE, col.names = FALSE)


Checking Inputs...

ArchR logging to : ArchRLogs/ArchR-addIterativeLSI-11dc38524e87-Date-2024-05-11_Time-23-41-15.log
If there is an issue, please report to github with logFile!

2024-05-11 23:41:17 : Computing Total Across All Features, 0.001 mins elapsed.

2024-05-11 23:41:21 : Computing Top Features, 0.052 mins elapsed.

###########
2024-05-11 23:41:23 : Running LSI (1 of 2) on Top Features, 0.092 mins elapsed.
###########

2024-05-11 23:41:23 : Creating Partial Matrix, 0.094 mins elapsed.

2024-05-11 23:41:38 : Computing LSI, 0.347 mins elapsed.

2024-05-11 23:42:13 : Identifying Clusters, 0.929 mins elapsed.

2024-05-11 23:42:23 : Identified 7 Clusters, 1.1 mins elapsed.

2024-05-11 23:42:23 : Saving LSI Iteration, 1.101 mins elapsed.

2024-05-11 23:42:44 : Creating Cluster Matrix on the total Group Features, 1.451 mins elapsed.

2024-05-11 23:43:39 : Computing Variable Features, 2.361 mins elapsed.

###########
2024-05-11 23:43:39 : Running LSI (2 of 2) on Variable Features, 2.36

Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 3413
Number of edges: 134360

Running Louvain algorithm...
Maximum modularity in 10 random starts: 0.8565
Number of communities: 13
Elapsed time: 0 seconds


2024-05-11 23:44:31 : Testing Outlier Clusters, 0.082 mins elapsed.

2024-05-11 23:44:31 : Assigning Cluster Names to 13 Clusters, 0.084 mins elapsed.

2024-05-11 23:44:31 : Finished addClusters, 0.087 mins elapsed.

23:44:32 UMAP embedding parameters a = 0.583 b = 1.334

23:44:32 Read 3413 rows and found 30 numeric columns

23:44:32 Using Annoy for neighbor search, n_neighbors = 30

23:44:32 Building Annoy index with metric = cosine, n_trees = 50

0%   10   20   30   40   50   60   70   80   90   100%

[----|----|----|----|----|----|----|----|----|----|

*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
*
|

23:44:32 Writing NN index file to temp file /scratch/local/RtmpcNpL1Z/file11dc342aee87e

23:44:32 Searching Annoy index using 64 threads, search_k = 3000

23:44:33 Annoy recall = 100%

23:44:35 Commencing smooth kNN distance calibration using 64 threads
 with target n_neighbors = 30

23:44:36 Initializing from normalized Laplacian + 

    Group nCells nCellsUsed nReplicates nMin nMax maxPeaks
C1     C1    610        540           2   40  500   150000
C2     C2    242        242           2   40  202   121000
C3     C3    276        276           2   40  236   138000
C4     C4    129        129           2   40   89    64500
C5     C5    256        256           2   40  216   128000
C6     C6     37         34           2   21   24    17000
C7     C7    207        207           2   40  167   103500
C8     C8    651        540           2   40  500   150000
C9     C9    130        130           2   40   90    65000
C10   C10    145        145           2   40  105    72500
C11   C11    325        325           2   40  285   150000
C12   C12    180        180           2   40  140    90000
C13   C13    225        225           2   40  185   112500


2024-05-12 00:26:30 : Batching Peak Calls!, 0.081 mins elapsed.

2024-05-12 00:26:31 : Batch Execution w/ safelapply!, 0 mins elapsed.

2024-05-12 00:26:31 : Group 1 of 24, Calling Peaks with MACS2!, 0.001 mins elapsed.

Running Macs2 with Params : macs2 callpeak -g 2.7e+09 --name C1._.Rep1-1 --treatment /project/scATAC_analysis/scATAcat_review/notebooks/Fig6-Granja_PBMC/Fig6-apply_Cellcano/BMMC_D6T1/PeakCalls/InsertionBeds/C1._.Rep1-1.insertions.bed --outdir /project/scATAC_analysis/scATAcat_review/notebooks/Fig6-Granja_PBMC/Fig6-apply_Cellcano/BMMC_D6T1/PeakCalls/InsertionBeds --format BED --call-summits --keep-dup all --nomodel --nolambda --shift -75 --extsize 150 -q 0.1

2024-05-12 00:28:04 : Group 2 of 24, Calling Peaks with MACS2!, 1.549 mins elapsed.

Running Macs2 with Params : macs2 callpeak -g 2.7e+09 --name C1._.Rep2-2 --treatment /project/scATAC_analysis/scATAcat_review/notebooks/Fig6-Granja_PBMC/Fig6-apply_Cellcano/BMMC_D6T1/PeakCalls/InsertionBeds/C1._.Rep2-2.insertions.b

...

[1] "/project/scATAC_analysis/scATAcat_review/notebooks/Fig6-Granja_PBMC/Fig6-apply_Cellcano/BMMC_D6T1/PeakCalls/C8-reproduciblePeaks.gr.rds"


Annotating Peaks : Nearest Gene

Annotating Peaks : Gene

Annotating Peaks : TSS

Annotating Peaks : GC



[1] "/project/scATAC_analysis/scATAcat_review/notebooks/Fig6-Granja_PBMC/Fig6-apply_Cellcano/BMMC_D6T1/PeakCalls/C9-reproduciblePeaks.gr.rds"


2024-05-12 00:47:48 : Creating Union Peak Set!, 21.381 mins elapsed.

Converged after 9 iterations!

Plotting Ggplot!

2024-05-12 00:47:54 : Finished Creating Union Peak Set (97474)!, 21.495 mins elapsed.

ArchR logging to : ArchRLogs/ArchR-addPeakMatrix-11dc335022f1f-Date-2024-05-12_Time-00-47-54.log
If there is an issue, please report to github with logFile!

2024-05-12 00:47:55 : Batch Execution w/ safelapply!, 0 mins elapsed.

2024-05-12 00:47:57 : Adding BMMC_D6T1 to PeakMatrix for Chr (1 of 23)!, 0.021 mins elapsed.

2024-05-12 00:48:03 : Adding BMMC_D6T1 to PeakMatrix for Chr (2 of 23)!, 0.125 mins elapsed.

2024-05-12 00:48:09 : Adding BMMC_D6T1 to PeakMatrix for Chr (3 of 23)!, 0.221 mins elapsed.

2024-05-12 00:48:14 : Adding BMMC_D6T1 to PeakMatrix for Chr (4 of 23)!, 0.309 mins elapsed.

2024-05-12 00:48:19 : Adding BMMC_D6T1 to PeakMatrix for Chr (5 of 23)!, 0.387 mins elapsed.

2024-05-12 00:48:24 : Adding BMMC_D6T1 to PeakMatrix for Chr (6 of 23)!, 0.471 mins elapsed.

2

NULL

In [ ]:
peakMatrix <- getMatrixFromProject(proj3, useMatrix='PeakMatrix')

PeakMatrix_proj3_dense =  as.matrix(peakMatrix@assays@data@listData$PeakMatrix)

write.csv(PeakMatrix_proj3_dense, file = paste(out_path,"PeakMatrix_dense.csv",sep="" )) 

# get the peak regions
proj3_peak_metadata  = (getPeakSet(proj3))
proj3_peak_regions = data.frame(seqnames = seqnames(proj3_peak_metadata),
           start= start(proj3_peak_metadata),
            end = end(proj3_peak_metadata))



In [17]:
getAvailableMatrices(proj3)

[1] "GeneScoreMatrix" "PeakMatrix"      "TileMatrix"

In [18]:
# get the gene activity matrix 

geneScoreMatrix <- getMatrixFromProject(proj3, useMatrix='GeneScoreMatrix')




ArchR logging to : ArchRLogs/ArchR-getMatrixFromProject-11dc348026e88-Date-2024-05-12_Time-10-40-27.log
If there is an issue, please report to github with logFile!

2024-05-12 10:40:40 : Organizing colData, 0.212 mins elapsed.

2024-05-12 10:40:40 : Organizing rowData, 0.213 mins elapsed.

2024-05-12 10:40:40 : Organizing rowRanges, 0.213 mins elapsed.

2024-05-12 10:40:40 : Organizing Assays (1 of 1), 0.214 mins elapsed.

2024-05-12 10:40:40 : Constructing SummarizedExperiment, 0.214 mins elapsed.

2024-05-12 10:40:41 : Finished Matrix Creation, 0.234 mins elapsed.



In [25]:
rowData(geneScoreMatrix)

DataFrame with 24919 rows and 6 columns
      seqnames     start       end  strand         name     idx
         <Rle>   <array>   <array> <array>      <array> <array>
1         chr1     69091     70008       1        OR4F5       1
2         chr1    140339    139790       2    LOC729737       2
3         chr1    629010    585990       2 LOC101928626       3
4         chr1    817371    819837       1       FAM87B       4
5         chr1    825138    859446       1    LINC01128       5
...        ...       ...       ...     ...          ...     ...
24915     chrX 155264589 155258241       2       RAB39B     952
24916     chrX 155334657 155276211       2        CLIC2     953
24917     chrX 155380787 155381134       1       H2AFB1     954
24918     chrX 155382115 155383230       1         F8A2     955
24919     chrX 155669944 155490115       2        TMLHE     956

In [26]:
dim(geneScoreMatrix)

[1] 24919  3413

In [31]:
geneScore_outDir = "/project/scATAC_analysis/scATAcat_review/data/Granja2019/BMMC_D6T1/05_peaks_and_genesScore_matrix_ArchR/"

In [36]:
write.table(rowData(geneScoreMatrix)$name, paste(geneScore_outDir, "ArchR_genescore_genes.tsv", sep=""),quote = FALSE, row.names=FALSE, col.names = FALSE)

In [37]:
write.table(colnames(geneScoreMatrix), paste(geneScore_outDir, "ArchR_genescore_barcodes.tsv", sep=""),quote = FALSE, row.names=FALSE, col.names = FALSE)

In [39]:
geneScoreMatrix_sparse = geneScoreMatrix@assays@data@listData$GeneScoreMatrix


In [40]:
writeMM(obj = geneScoreMatrix_sparse, file=paste(geneScore_outDir, "ArchR_genescore.mtx", sep=""))

NULL

In [43]:
saveArchRProject(ArchRProj = proj3, outputDirectory = paste(geneScore_outDir,"Save-Proj", sep=""), load = FALSE)

Copying ArchRProject to new outputDirectory : /project/scATAC_analysis/scATAcat_review/data/Granja2019/BMMC_D6T1/05_peaks_and_genesScore_matrix_ArchR/Save-Proj

Copying Arrow Files...

Copying Arrow Files (1 of 1)

Getting ImputeWeights

No imputeWeights found, returning NULL

Copying Other Files...

Copying Other Files (1 of 5): Embeddings

Copying Other Files (2 of 5): GroupCoverages

Copying Other Files (3 of 5): IterativeLSI

Copying Other Files (4 of 5): PeakCalls

Copying Other Files (5 of 5): Plots

Saving ArchRProject...



In [58]:
library(stringr)

write.table(as.data.frame(str_split_fixed(colnames(geneScoreMatrix),'#',2))$V2, paste(geneScore_outDir, "cell_IDs.tsv", sep=""),quote = FALSE, row.names=FALSE, col.names = FALSE)
